In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import ScalarFormatter # Useful for various plotting, though not strictly for ER here

# --- Helper Function to Calculate Engagement Rate ---
def calculate_engagement_rate(row, sub_col_name, likes_col_name, comments_col_name, shares_col_name=None):
    """
    Calculates a simplified engagement rate based on average interactions per post/video.
    Formula: (Average Likes + Average Comments + Average Shares) / Subscribers * 100
    Handles potential NaN values for interaction metrics (treats as 0) and division by zero subscribers.

    Args:
        row (pd.Series): A row from the DataFrame.
        sub_col_name (str): The actual column name for subscribers/followers.
        likes_col_name (str): The actual column name for average likes.
        comments_col_name (str): The actual column name for average comments.
        shares_col_name (str, optional): The actual column name for average shares (e.g., for TikTok). Defaults to None.

    Returns:
        float: The calculated engagement rate as a percentage, or NaN if calculation is not possible.
    """
    subscribers = row.get(sub_col_name)
    likes = row.get(likes_col_name, 0)
    comments = row.get(comments_col_name, 0)
    shares = row.get(shares_col_name, 0) if shares_col_name else 0

    # Ensure numeric and handle NaNs for calculation
    subscribers = pd.to_numeric(subscribers, errors='coerce')
    likes = pd.to_numeric(likes, errors='coerce')
    comments = pd.to_numeric(comments, errors='coerce')
    shares = pd.to_numeric(shares, errors='coerce')

    likes = 0 if pd.isna(likes) else likes
    comments = 0 if pd.isna(comments) else comments
    shares = 0 if pd.isna(shares) else shares

    if pd.isna(subscribers) or subscribers == 0:
        return np.nan # Cannot calculate ER without valid subscriber count

    total_interactions = likes + comments + shares

    return (total_interactions / subscribers) * 100

# --- Main Analysis Function for Engagement Rate ---
def analyze_engagement_rate(insta_df, tiktok_df, youtube_df,
                            insta_col_map={'subscribers': 'subscribers', 'likes_avg': 'likes_avg', 'comments_avg': 'comments_avg'},
                            tiktok_col_map={'subscribers': 'subscribers', 'likes_avg': 'likes_avg', 'comments_avg': 'comments_avg', 'shares_avg': 'shares_avg'},
                            yt_col_map={'subscribers': 'subscribers', 'likes_avg': 'avg_likes', 'comments_avg': 'avg_comments'}):
    """
    Calculates and visualizes engagement rates for influencers on Instagram, TikTok, and YouTube.

    Args:
        insta_df (pd.DataFrame): DataFrame containing Instagram influencer data.
        tiktok_df (pd.DataFrame): DataFrame containing TikTok influencer data.
        youtube_df (pd.DataFrame): DataFrame containing YouTube influencer data.
        insta_col_map (dict): Dictionary mapping generic metric names (keys) to actual column names (values) for Instagram.
        tiktok_col_map (dict): Dictionary mapping generic metric names to actual column names for TikTok.
        yt_col_map (dict): Dictionary mapping generic metric names to actual column names for YouTube.
    """
    print("--- Starting Engagement Rate Analysis ---")

    all_engagement_data = [] # To store engagement rates for all platforms

    # Define configurations for each platform
    platforms_config = {
        'Instagram': {'df': insta_df, 'col_map': insta_col_map, 'include_shares': False},
        'TikTok':    {'df': tiktok_df, 'col_map': tiktok_col_map, 'include_shares': True},
        'YouTube':   {'df': youtube_df, 'col_map': yt_col_map, 'include_shares': False}
    }

    for platform_name, config in platforms_config.items():
        df = config['df']
        col_map = config['col_map']
        include_shares = config['include_shares']

        if df is None:
            print(f"\nSkipping {platform_name}: DataFrame is None (check file path).")
            continue

        # Get actual column names from the mapping
        sub_col = col_map.get('subscribers')
        likes_col = col_map.get('likes_avg')
        comments_col = col_map.get('comments_avg')
        shares_col = col_map.get('shares_avg') if include_shares else None

        # Check if all absolutely required columns exist in the DataFrame
        required_present_cols = [col for col in [sub_col, likes_col, comments_col] if col is not None and col in df.columns]
        expected_required_cols_count = 3 # subscribers, likes_avg, comments_avg

        if len(required_present_cols) < expected_required_cols_count:
            missing_for_platform = [col for col in [sub_col, likes_col, comments_col] if col not in df.columns]
            print(f"\nSkipping {platform_name}: Missing one or more required columns for ER calculation: {missing_for_platform} or mapping is incorrect.")
            continue

        # Prepare DataFrame for calculation: select only necessary columns
        cols_for_calc = [sub_col, likes_col, comments_col]
        if shares_col and shares_col in df.columns:
            cols_for_calc.append(shares_col)
        else:
            shares_col = None # Ensure it's None if not present/mapped for this platform

        df_for_er = df[cols_for_calc].copy()

        # Calculate engagement rate using the helper function
        df_for_er['engagement_rate'] = df_for_er.apply(
            lambda row: calculate_engagement_rate(row, sub_col, likes_col, comments_col, shares_col),
            axis=1
        )

        # Add platform column and append to overall list
        df_for_er['platform'] = platform_name
        valid_er_data = df_for_er.dropna(subset=['engagement_rate'])

        if not valid_er_data.empty:
            all_engagement_data.append(valid_er_data[['engagement_rate', 'platform']])
            print(f"\n--- {platform_name} Engagement Rate Summary ({valid_er_data.shape[0]} valid entries) ---")
            print(valid_er_data['engagement_rate'].describe().to_string())
        else:
            print(f"\nNo valid engagement rate data for {platform_name} after calculation and cleaning.")


    # --- Combined Engagement Rate Visualizations ---
    if all_engagement_data:
        combined_er_df = pd.concat(all_engagement_data, ignore_index=True)

        if not combined_er_df.empty:
            print("\n--- Visualizing Engagement Rates Across Platforms ---")

            # --- Box Plot of Engagement Rate Distribution ---
            plt.figure(figsize=(10, 7))
            sns.boxplot(x='platform', y='engagement_rate', data=combined_er_df, palette='viridis')
            plt.title('Engagement Rate Distribution Across Platforms')
            plt.xlabel('Platform')
            plt.ylabel('Engagement Rate (%)')
            # Set y-axis limit to focus on the majority of data, ignoring extreme outliers
            # You can adjust the percentile (e.g., 0.995) or multiplier (e.g., 1.5) as needed
            plt.ylim(0, combined_er_df['engagement_rate'].quantile(0.99) * 1.2)
            plt.grid(axis='y', linestyle='--', alpha=0.7)
            plt.tight_layout()
            plt.show()

            # --- Violin Plot of Engagement Rate Distribution ---
            plt.figure(figsize=(10, 7))
            sns.violinplot(x='platform', y='engagement_rate', data=combined_er_df, palette='viridis')
            plt.title('Detailed Engagement Rate Distribution Across Platforms')
            plt.xlabel('Platform')
            plt.ylabel('Engagement Rate (%)')
            plt.ylim(0, combined_er_df['engagement_rate'].quantile(0.99) * 1.2) # Apply same limit
            plt.grid(axis='y', linestyle='--', alpha=0.7)
            plt.tight_layout()
            plt.show()

            # --- Bar Plot of Average Engagement Rate ---
            plt.figure(figsize=(9, 6))
            avg_er = combined_er_df.groupby('platform')['engagement_rate'].mean().sort_values(ascending=False)
            sns.barplot(x=avg_er.index, y=avg_er.values, palette='coolwarm')
            plt.title('Average Engagement Rate by Platform')
            plt.xlabel('Platform')
            plt.ylabel('Average Engagement Rate (%)')
            plt.grid(axis='y', linestyle='--', alpha=0.7)
            plt.tight_layout()
            plt.show()

        else:
            print("No valid engagement rate data could be consolidated for visualization. Please check data and column mappings.")
    else:
        print("No engagement rate data could be processed for any platform. Check input DataFrames and column mappings.")

    print("\n--- Engagement Rate Analysis Complete ---")


# --- EXAMPLE DATA LOADING (REPLACE WITH YOUR ACTUAL DATA) ---
# IMPORTANT:
# 1. Place your CSV files in the same directory as this Python script,
#    OR provide the full path to your files (e.g., 'C:/Users/YourUser/Desktop/instagram_data.csv').
# 2. ENSURE THE '.csv' EXTENSION IS CORRECT (e.g., '.xlsx' if it's an Excel file).

try:
    # Ensure these columns exist in your Instagram data
    insta_df = pd.read_csv('./Clean_Data/merged_instagram_data_cleaned.csv')
    print("Instagram data loaded.")
except FileNotFoundError:
    print("instagram_data.csv not found. Please check the file path.")
    insta_df = None

try:
    # Ensure these columns exist in your TikTok data
    tiktok_df = pd.read_csv('./Clean_Data/merged_tiktok_data_cleaned.csv')
    print("TikTok data loaded.")
except FileNotFoundError:
    print("tiktok_data.csv not found. Please check the file path.")
    tiktok_df = None

try:
    # Ensure these columns exist in your YouTube data
    youtube_df = pd.read_csv('./Clean_Data/merged_youtube_data_cleaned.csv')
    print("YouTube data loaded.")
except FileNotFoundError:
    print("youtube_data.csv not found. Please check the file path.")
    youtube_df = None


# --- CALL THE ENGAGEMENT RATE ANALYSIS FUNCTION ---
# VERY IMPORTANT:
# Adjust the column names in the dictionaries below to EXACTLY match the column
# headers in YOUR specific CSV files.
#
# Common column name examples:
# - Subscribers: 'subscribers', 'followers', 'audience_size'
# - Average Likes: 'likes_avg', 'average_likes_per_post', 'avg_hearts'
# - Average Comments: 'comments_avg', 'average_comments_per_post'
# - Average Shares: 'shares_avg', 'average_shares_per_post' (primarily for TikTok)

analyze_engagement_rate(
    insta_df=insta_df,
    tiktok_df=tiktok_df,
    youtube_df=youtube_df,
    # --- Instagram Column Mappings ---
    insta_col_map={
        'subscribers': 'subscribers',   # <--- Change this if your Instagram subscriber column is different
        'likes_avg': 'likes_avg',       # <--- Change this if your Instagram avg likes column is different
        'comments_avg': 'comments_avg'  # <--- Change this if your Instagram avg comments column is different
    },
    # --- TikTok Column Mappings ---
    tiktok_col_map={
        'subscribers': 'subscribers',   # <--- Change this if your TikTok subscriber column is different
        'likes_avg': 'likes_avg',       # <--- Change this if your TikTok avg likes column is different
        'comments_avg': 'comments_avg', # <--- Change this if your TikTok avg comments column is different
        'shares_avg': 'shares_avg'      # <--- Change this if your TikTok avg shares column is different (set to None in func call if not present)
    },
    # --- YouTube Column Mappings ---
    yt_col_map={
        'subscribers': 'subscribers',   # <--- Change this if your YouTube subscriber column is different
        'likes_avg': 'avg_likes',       # <--- Change this if your YouTube avg likes column is different (e.g., 'likes_per_video')
        'comments_avg': 'avg_comments'  # <--- Change this if your YouTube avg comments column is different (e.g., 'comments_per_video')
    }
)

Instagram data loaded.
TikTok data loaded.
YouTube data loaded.
--- Starting Engagement Rate Analysis ---

No valid engagement rate data for Instagram after calculation and cleaning.

No valid engagement rate data for TikTok after calculation and cleaning.

No valid engagement rate data for YouTube after calculation and cleaning.
No engagement rate data could be processed for any platform. Check input DataFrames and column mappings.

--- Engagement Rate Analysis Complete ---


In [2]:
# --- Helper Function for Data Inspection (Add this to your script) ---
def inspect_dataframe_for_engagement_rate(df, platform_name, col_map):
    """
    Helps inspect a DataFrame for common issues preventing engagement rate calculation.
    """
    print(f"\n--- INSPECTING {platform_name} DataFrame ---")
    if df is None:
        print(f"DataFrame for {platform_name} is None. Cannot inspect.")
        return

    print("\n--- First 5 rows of DataFrame ---")
    print(df.head().to_string()) # Use .to_string() for better console display

    print("\n--- DataFrame Info (Column Names & Data Types) ---")
    df.info()

    print("\n--- Checking Specific Columns for Engagement Rate Calculation ---")
    sub_col = col_map.get('subscribers')
    likes_col = col_map.get('likes_avg')
    comments_col = col_map.get('comments_avg')
    shares_col = col_map.get('shares_avg') # May be None

    columns_to_check = {
        'subscribers': sub_col,
        'likes_avg': likes_col,
        'comments_avg': comments_col,
        'shares_avg': shares_col # Only if it's expected for this platform
    }

    actual_present_cols = []
    print(f"Expected Column Mappings for {platform_name}:")
    for generic_name, actual_name in columns_to_check.items():
        if actual_name: # Only check if a column name was provided in the map
            print(f"  - {generic_name} expects actual column: '{actual_name}'")
            if actual_name in df.columns:
                print(f"    - '{actual_name}' FOUND in DataFrame columns.")
                actual_present_cols.append(actual_name)
                # Check for non-numeric values
                original_count = df[actual_name].shape[0]
                numeric_series = pd.to_numeric(df[actual_name], errors='coerce')
                nan_count = numeric_series.isna().sum()
                print(f"    - '{actual_name}' has {nan_count} non-numeric/missing values out of {original_count} total.")
                if nan_count > 0 and nan_count == original_count:
                    print(f"      CRITICAL: All values in '{actual_name}' are non-numeric/missing. This column will be entirely NaN after conversion.")
                elif nan_count > 0:
                     print(f"      WARNING: Some values in '{actual_name}' are non-numeric/missing. Inspect these rows.")

            else:
                print(f"    - CRITICAL: Column '{actual_name}' (for {generic_name}) NOT FOUND in DataFrame columns. Please check your mapping.")
        else:
            print(f"  - No mapping provided for {generic_name}.")

    if not actual_present_cols:
        print(f"\nNo relevant engagement rate columns found for {platform_name}. Double-check your `col_map` and DataFrame headers.")


# --- Place this section in your existing script ---
# After your try-except blocks for loading insta_df, tiktok_df, youtube_df
# and BEFORE the call to analyze_engagement_rate(...)

# --- Data Inspection Calls ---
# IMPORTANT: Use the SAME col_map dictionaries you use in the analyze_engagement_rate function call.
print("\n===== Running Data Inspection =====")
inspect_dataframe_for_engagement_rate(insta_df, 'Instagram', {
    'subscribers': 'subscribers',
    'likes_avg': 'likes_avg',
    'comments_avg': 'comments_avg'
})
inspect_dataframe_for_engagement_rate(tiktok_df, 'TikTok', {
    'subscribers': 'subscribers',
    'likes_avg': 'likes_avg',
    'comments_avg': 'comments_avg',
    'shares_avg': 'shares_avg'
})
inspect_dataframe_for_engagement_rate(youtube_df, 'YouTube', {
    'subscribers': 'subscribers',
    'likes_avg': 'avg_likes',
    'comments_avg': 'avg_comments'
})
print("\n===== Data Inspection Complete =====")

# --- Your existing analyze_engagement_rate() call will go here ---
# analyze_engagement_rate(...)


===== Running Data Inspection =====

--- INSPECTING Instagram DataFrame ---

--- First 5 rows of DataFrame ---
   sno instagram_name               name  subscribers audience_country authentic_engagement engagement_average                 category_1 category_2 influencer_insta_name audience_country(mostly) engagement_avg   rank eng_(auth) eng_(avg) country influencer_name subscribers_count views_avg likes_avg comments_avg
0  1.0      cristiano  Cristiano Ronaldo            0            India                   5M               6.2M         Sports with a ball    Unknown                   NaN                      NaN            NaN  499.0        NaN       NaN     NaN             NaN               NaN       NaN       NaN          NaN
1  2.0    kyliejenner            Kylie 🤍            0    United States                 3.5M               5.5M                    Fashion   Modeling                   NaN                      NaN            NaN  499.0        NaN       NaN     NaN             N